## Population Demography
### ROH length and estimates of recent N<sub>e</sub>
The below code and method was adapted from [Humble et al. (2023)](http://doi.org/10.1073/pnas.2210756120) to estimate recent trends in N<sub>e</sub> with runs of homozygosity (ROH). The original code from this paper is available [here](https://github.com/ehumble/SHO_roh_load_2022/tree/master).  

The formula below summarised by Humble et al from [Thompson (2013)](https://doi.org/10.1534/genetics.112.148825) was used to divide ROHs into seven length classes (*L*) corresponding to <2, 2-4, 4-8, 8-16, 16-32, 32-64, and 64-128 generations (*g*) ago:
$$
L = \frac{100}{2 * g}
$$

Essentially, this equation is assuming that a time of depth of *g* should result in double the number of meioses events (*m*). Given that a linkage map was unavailable for their study species, Humble et al assumed a recombination rate of 1 cM/Mb. However, recombination rate is generally higher than this estimate and has been shown to be heterogenous within and between avian lineages and even within and between chromosomes ([Nam et al. (2010)](https://doi.org/10.1186/gb-2010-11-6-r68)). To get an understanding of the broad patterns of N<sub>e</sub>, we estimated N<sub>e</sub> using a mean recombination rate of 2.8 cM/Mb as a middle road of recombination rates observed in [Backström et al. (2010)](http://doi.org/10.1101/gr.101410.109) and [Smeds et al. (2016)](http://doi.org/10.1371/journal.pgen.1006044) and as it was estimated in the chicken (*Gallus gallus*).
$$
L = \frac{100}{2.8(2 * g)}
$$

Using the lengths calculated here, we estimated N<sub>e</sub> for each *g*. Here, *F<sub>ROH, t</sub>* represents average population inbreeding coefficient for the maximum numbers of generations (*t*) in the time periods corresponding for each of the seven size classes above.  
$$
F_{ROH, t} = 1 - (1 - \frac{1}{2 * N_{e}})^t
$$

We then define the function `inb_ne()` to estimate N<sub>e</sub> using the mean *F<sub>ROH</sub>* for each *L*.  
$$
N_{e} = \frac{1}{2 * (1 - f_{ROH})^\frac{1}{L}}
$$

Below we also define a function (`bootstrap_Ne()`) for bootstrapping with replacement for each N<sub>e</sub> estimate.

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

#path = '/nesi/nobackup/uc03718/'
#os.chdir(path)
print(os.getcwd())

: 

In [ ]:
def inb_ne(mean_froh, class_):
    Ne = 1 / (2 * (1 - ((1 - mean_froh) ** (1 / class_))))
    return Ne

def bootstrap_Ne(iter, df, frac):
    sampled_df = (df
                  .groupby(['Generation Class', 'Population'], group_keys=False)
                  .apply(lambda x: x.sample(frac=frac, replace=True))
                  .groupby(['Generation Class', 'Population'], as_index=False)
                  .agg(mean_froh = ('fROH', 'mean'))
                  .assign(Ne = lambda x: inb_ne(x['mean_froh'], x['Generation Class']))
                  )
    return sampled_df

To estimate N<sub>e</sub> for each of the seven *L* defined above, we us the `ROHs.tsv` file. This time, we convert ROH sizes estimated by ROHAN to megabases (defined as `ROH Size in Mb` in the df below), assign the relevant Generation Class (*g*) for each *L* (defined as `length_class`) and find the *F<sub>ROH</sub>* for each ROH in each individual by dividing `ROH Size in Mb` by the estimated size of autosomal chromosomes for the tara iti and kakī reference genomes (in Mb).

Given the influence relatedness among individuals may have on estimates of contemporary N<sub>e</sub>, we excluded all offspring in kakī parental/offspring trios in our data set. We found did not observe obvious differences in our N<sub>e</sub> estimates from parents only vs the entire dataset.  

In [ ]:
roh = pd.read_csv('ROHAN_out/ROHs.tsv', delimiter='\t')

roh['ROH Size in Mb'] = roh['ROH_size'] / 1000000

recomb = roh
recomb['Generation Class'] = np.select(
    [
        recomb['ROH Size in Mb'] >= 17.8571,
        (recomb['ROH Size in Mb'] < 17.8571) & (recomb['ROH Size in Mb'] >= 4.4643),
        (recomb['ROH Size in Mb'] < 4.4643) & (recomb['ROH Size in Mb'] >= 2.2321),
        (recomb['ROH Size in Mb'] < 2.2321) & (recomb['ROH Size in Mb'] >= 1.1161),
        (recomb['ROH Size in Mb'] < 1.1161) & (recomb['ROH Size in Mb'] >= 0.5580),
        (recomb['ROH Size in Mb'] < 0.5580) & (recomb['ROH Size in Mb'] >= 0.2790),
        (recomb['ROH Size in Mb'] < 0.2790) & (recomb['ROH Size in Mb'] >= 0.1395),
    ],
    [2, 4, 8, 16, 32, 64, 128],
    default=np.nan
)

recomb['length_class'] = recomb['Generation Class'].map({
    2: ">17.8571",
    4: "4.4643-17.8571",
    8: "2.2321-4.4643",
    16: "1.1161-2.2321",
    32: "0.5580-1.1161",
    64: "0.2790-0.5580",
    128: "0.1395-0.2790"
})

fairy_recomb = recomb[(recomb['Population']!='KI') & (recomb['Population']!='KI_10x') & (recomb['ROH Size in Mb']>=0.1395)]
#kakiLC_recomb = recomb[(recomb['Population']=='KI_10x') & (recomb['ROH Size in Mb']>=0.1395)]
#kakiLC_recomb = kakiLC_recomb[kakiLC_recomb['Sample'].isin(['H01383', 'H01384', 'H01385', 'H01386', 'H01387', 'H01388', 'H01389', 'H01390', 'H01391', 'H01392', 'H01393', 'H01394'])]
#kaki_recomb = recomb[(recomb['Population']=='KI') & (recomb['ROH Size in Mb']>=0.1395)]
#kaki_recomb = kaki_recomb[kaki_recomb['Sample'].isin(['H01383', 'H01384', 'H01385', 'H01386', 'H01387', 'H01388', 'H01389', 'H01390', 'H01391', 'H01392', 'H01393', 'H01394'])]

fairy = (fairy_recomb
    .groupby(['Sample', 'Population', 'Generation Class', 'length_class'], as_index=False, dropna=False)
    .agg(fROH=('ROH Size in Mb', lambda x: sum((x / 1088.797119))))
    )

#kaki = (kaki_recomb
#    .groupby(['Sample', 'Population', 'Generation Class', 'length_class'], as_index=False, dropna=False)
#    .agg(fROH=('ROH Size in Mb', lambda x: sum((x / 1095.624494))))
#    )

#kakiLC = (kakiLC_recomb
#    .groupby(['Sample', 'Population', 'Generation Class', 'length_class'], as_index=False, dropna=False)
#    .agg(fROH=('ROH Size in Mb', lambda x: sum((x / 1095.624494))))
#    )

fairy.head()

In [ ]:
iterations = 10000
#kiresults = [bootstrap_Ne(i, kaki, 0.5) for i in range(iterations)]
#kiLCresults = [bootstrap_Ne(i, kakiLC, 0.5) for i in range(iterations)]
ftresults = [bootstrap_Ne(i, fairy, 0.5) for i in range(iterations)]

#kiuncert = (pd.concat(kiresults, keys=range(iterations), names=['iter'])
#                .reset_index(level='iter'))

#kiLCuncert = (pd.concat(kiLCresults, keys=range(iterations), names=['iter'])
#                .reset_index(level='iter'))

ftuncert = (pd.concat(ftresults, keys=range(iterations), names=['iter'])
                .reset_index(level='iter'))

#uncertainty = pd.concat([ftuncert, kiLCuncert, kiuncert], axis=0, ignore_index=True)

In [ ]:
print(ftuncert.groupby(['Generation Class', 'Population'])['Ne'].min())

In [ ]:
def calculate_SE(group):
    std_dev = group['Ne'].std()
    count = group['Ne'].count()
    standard_error = std_dev / np.sqrt(count)
    group['SE'] = standard_error
    group['SD'] = std_dev
    return group

Ne = uncertainty.groupby(['Generation Class', 'Population']).apply(calculate_SE).reset_index(drop=True)
Ne = Ne.groupby(['Generation Class', 'Population', 'SD', 'SE'])['Ne'].mean().reset_index()
Ne.head(n=20)

In [ ]:
palette = ['gold', 'steelblue', 'grey', 'black']
plt.figure(figsize=(15, 5))
ax = sns.pointplot(data=uncertainty[uncertainty['Generation Class']>2], x='Generation Class', y='Ne', dodge=True, hue='Population', hue_order=['AU', 'TI', 'KI_10x', 'KI'],
                palette=palette, errorbar='sd', capsize=0.05, n_boot=100, estimator='mean', markers='o', linestyles='none')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles,
        labels=['AFT', 'TI', 'KĪ low', 'KĪ high'],
        loc='upper left', handlelength=4,
        handler_map={tuple: HandlerTuple(ndivide=None)})

params = {'mathtext.default': 'regular' }
plt.rcParams.update(params)
ax.set_title('', loc='left', fontsize=20)
ax.set_xticklabels(['2-4', '4-8', '8-16', '16-32', '32-64', '64-128'])
ax.set_xlabel('Generations', fontsize=14)
ax.set_ylabel('$N_{e}$', fontsize=14)

#plt.savefig('plots/ROH_Recomb_Ne_esimtates.png', dpi=300, bbox_inches='tight')

In [ ]:
palette = ['steelblue', 'grey', 'black']

uncertainty_subplot = uncertainty[(uncertainty['Generation Class']>2) & (uncertainty['Generation Class']<128) & (uncertainty['Population']!='AU')]

plt.figure(figsize=(15, 5))
ax = sns.pointplot(data=uncertainty_subplot, x='Generation Class', y='Ne', dodge=True, hue='Population', hue_order=['TI', 'KI_10x', 'KI'],
                palette=palette, errorbar='sd', capsize=0.05, n_boot=100, estimator='mean', markers='o', linestyles='none')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles,
        labels=['TI', 'KĪ low', 'KĪ'],
        loc='upper left', handlelength=4,
        handler_map={tuple: HandlerTuple(ndivide=None)})

params = {'mathtext.default': 'regular' }
plt.rcParams.update(params)
ax.set_title('', loc='left', fontsize=20)
ax.set_xticklabels(['2-4', '4-8', '8-16', '16-32', '32-64'])
ax.set_xlabel('Generations', fontsize=14)
ax.set_ylabel('$N_{e}$', fontsize=14)

#plt.savefig('plots/ROH_Recomb_Ne_Subset_esimtates.png', dpi=300, bbox_inches='tight')

In [ ]:
gen2 = 17857100
gen4 = 4464300
gen8 = 2232100
gen16 = 1116100
large_roh = roh[(roh['ROH_size']>=gen4) & (roh['ROH_size']< gen2)]
large_roh = large_roh.groupby('Sample')['Population'].value_counts().reset_index()
large_roh = large_roh.groupby('Population')['count'].mean().reset_index()

huge_roh = roh[roh['ROH_size']>=gen2]
huge_roh = huge_roh.groupby('Sample')['Population'].value_counts().reset_index()
huge_roh = huge_roh.groupby('Population')['count'].mean().reset_index()

print(large_roh)

In [ ]:
roh[roh['ROH_size']>=gen2].groupby(['Sample','Population'])['Population'].count()

### StairwayPlot2
Below script was modified from [here](https://github.com/xiaoming-liu/stairway-plot-v2/issues/2).

In [ ]:
au_stairway = pd.read_csv('stairwayplot2/AU_neutral_temp/Australian fairy tern (neutral).final.summary', delimiter='\t')
ti_stairway = pd.read_csv('stairwayplot2/TI_neutral_temp/Tara iti (neutral).final.summary', delimiter='\t')
ki_stairway = pd.read_csv('stairwayplot2/KI_neutral_temp/Kakī (neutral).final.summary', delimiter='\t')

fig, axs = plt.subplots(3, 1, figsize=(20,10), sharex=True, sharey=False)
axs[0].step(au_stairway['year'], au_stairway['Ne_median'], where='post', color = 'gold', linewidth=1, label='Australia')
axs[0].step(au_stairway['year'], au_stairway['Ne_2.5%'], where='post', color = 'gold', linewidth=1, linestyle='dashed')
axs[0].step(au_stairway['year'], au_stairway['Ne_97.5%'], where='post', color = 'gold', linewidth=1, linestyle='dashed')
axs[0].set_title('A)', fontsize=20, loc='left')
axs[0].set_ylim(0, 100000)

axs[1].step(ti_stairway['year'], ti_stairway['Ne_median'], where='post', color = 'steelblue', linewidth=1, label='Tara iti')
axs[1].step(ti_stairway['year'], ti_stairway['Ne_2.5%'], where='post', color = 'steelblue', linewidth=1, linestyle='dashed')
axs[1].step(ti_stairway['year'], ti_stairway['Ne_97.5%'], where='post', color = 'steelblue', linewidth=1, linestyle='dashed')
axs[1].set_title('B)', fontsize=20, loc='left')
axs[1].set_ylim(0, 12500)

axs[2].step(ki_stairway['year'], ki_stairway['Ne_median'], where='post', color = 'black', linewidth=1, label='Kakī')
axs[2].step(ki_stairway['year'], ki_stairway['Ne_2.5%'], where='post', color = 'black', linewidth=1, linestyle='dashed')
axs[2].step(ki_stairway['year'], ki_stairway['Ne_97.5%'], where='post', color = 'black', linewidth=1, linestyle='dashed')
axs[2].set_title('C)', fontsize=20, loc='left')
axs[2].set_ylim(0, 25000)

plt.xlim(0, 800)

plt.xlabel('Years Before Present', fontsize=14)
plt.ylabel('Effective Population Size', fontsize=14)
#plt.xticks(x_ticks, x_labels, fontsize=10)
#plt.yticks(y_ticks, y_labels, fontsize=10)

plt.savefig('plots/fairy_Ne_plot2.png', dpi=300, bbox_inches='tight')

In [ ]:
au_stairway.tail()

In [ ]:
au_hgone = pd.read_csv('GONE/Haldane_Correction/AU_neutral_run1_test/Output_Ne_AU_neutral_test', sep='\t')
ki_hgone = pd.read_csv('GONE/Haldane_Correction/KI_neutral_run1/Output_Ne_KI_neutral', sep='\t')
ti_hgone = pd.read_csv('GONE/Haldane_Correction/TI_neutral_run1_test/Output_Ne_TI_neutral_test', sep='\t')

au_kgone = pd.read_csv('GONE/Kosambi_Correction/AU_neutral_run1/Output_Ne_AU_neutral_test', sep='\t')
ki_kgone = pd.read_csv('GONE/Kosambi_Correction/KI_neutral_run1/Output_Ne_KI_neutral', sep='\t')
ti_kgone = pd.read_csv('GONE/Kosambi_Correction/TI_neutral_run1/Output_Ne_TI_neutral_test', sep='\t')

ki_h_parents = pd.read_csv('GONE/Haldane_Correction/KI_neutral_parents/Output_Ne_KI_parents_neutral', sep='\t')

# Filter to within the last 100 generations as this is the noly period the method is confident in
au_hgone = au_hgone[au_hgone['Generation']<=100]
ki_hgone = ki_hgone[ki_hgone['Generation']<=100]
ti_hgone = ti_hgone[ti_hgone['Generation']<=100]

au_kgone = au_kgone[au_kgone['Generation']<=100]
ki_kgone = ki_kgone[ki_kgone['Generation']<=100]
ti_kgone = ti_kgone[ti_kgone['Generation']<=100]

ki_h_parents = ki_h_parents[ki_h_parents['Generation']<=100]
ki_h_parents['Year'] = ki_h_parents['Generation'] * 6

au_hgone['Population'] = 'AU'
ki_hgone['Population'] = 'KI'
ti_hgone['Population'] = 'TI'

au_hgone['Year'] = au_hgone['Generation'] * 6
ki_hgone['Year'] = ki_hgone['Generation'] * 6
ti_hgone['Year'] = ti_hgone['Generation'] * 6

au_kgone['Population'] = 'AU'
ki_kgone['Population'] = 'KI'
ti_kgone['Population'] = 'TI'

ki_h_parents['Population'] = 'KI'

au_hgone.head()

In [ ]:
sns.lineplot(ti_hgone, x='Year', y='Geometric_mean')
sns.lineplot(ki_hgone, x='Year', y='Geometric_mean')
sns.lineplot(ki_h_parents, x='Year', y='Geometric_mean', color='black')

In [ ]:
sns.lineplot(ti_hgone[ti_hgone['Generation']>10], x='Year', y='Geometric_mean')
sns.lineplot(ki_hgone[ki_hgone['Generation']>10], x='Year', y='Geometric_mean')
sns.lineplot(ki_h_parents[ki_h_parents['Generation']>10], x='Year', y='Geometric_mean', color='black')

In [ ]:
sns.lineplot(au_hgone[au_hgone['Generation']>10], x='Year', y='Geometric_mean', color='gold')